# Self-Driving Car Engineer Nanodegree


## Project: **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

In addition to implementing code, there is a brief writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) that can be used to guide the writing process. Completing both the code in the Ipython notebook and the writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/322/view) for this project.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note: If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".**

---

## Import Packages

In [1]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
from scipy import stats
import math
import os
%matplotlib inline

In [2]:
import imageio 
imageio.plugins.ffmpeg.download()

## Helper Functions

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [3]:
def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def fit_parabola(x, y):
    z = np.polyfit(x, y, 2)
    p = np.poly1d(z)
    z_apex_x = -(z[1]/(2*z[0]))
    return (z_apex_x, p(z_apex_x))

# Python 3 has support for cool math symbols.

def weighted_img(initial_img, img, α=0.7, β=0.8, λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

**Note: if you get an import error when you run the next cell, try changing your kernel (select the Kernel menu above --> Change Kernel). Still have problems? Try relaunching Jupyter Notebook from the terminal prompt. Also, consult the forums for more troubleshooting tips.**

**If you get an error that looks like this:**
```
NeedDownloadError: Need ffmpeg exe. 
You can download it by calling: 
imageio.plugins.ffmpeg.download()
```
**Follow the instructions in the error message and check out [this forum post](https://discussions.udacity.com/t/project-error-of-test-on-videos/274082) for more troubleshooting tips across operating systems.**

In [4]:
def get_fitLane(left_slope, left_intercept, right_slope, right_intercept, Y_lowerbound, Y_upperbound):
    return ((Y_lowerbound - left_intercept)/left_slope
            , (Y_upperbound - left_intercept)/left_slope
            , (Y_upperbound - right_intercept)/right_slope
            , (Y_lowerbound - right_intercept)/right_slope)
    
def get_fitLineCharacter(x, y):
    """
    format is y = slope * x + intercept
    """
    slope, intercept, _, _, std_err = stats.linregress(x,y)
    return slope, intercept

def draw_lines(img, lines, color=[255, 0, 0], thickness=10):
    for x1, y1, x2, y2 in lines:
        cv2.line(img, (int(x1), int(y1)), (int(x2), int(y2)), color, thickness)

def draw_circle(img, center, radius, color, thickness = 5):
    return cv2.circle(img, center, radius, color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap, filter_y_uppderbound):
    lines = np.squeeze(cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap), axis=1)

    pos_SlopeLines, neg_SlopeLines, left_center, right_center = hough_line_filter(lines.T, filter_y_uppderbound//2)
        
    Lane_Y_lowerbound = min(left_center[1], right_center[1])
    Lane_Y_upperbound = filter_y_uppderbound
    
    left_x = np.concatenate((neg_SlopeLines[0], neg_SlopeLines[2]), axis=0)
    left_y = np.concatenate((neg_SlopeLines[1], neg_SlopeLines[3]), axis=0)
    
    right_x = np.concatenate((pos_SlopeLines[0], pos_SlopeLines[2]), axis=0)
    right_y = np.concatenate((pos_SlopeLines[1], pos_SlopeLines[3]), axis=0)

    left_slope, left_intercept = get_fitLineCharacter(left_x, left_y)
    right_slope, right_intercept = get_fitLineCharacter(right_x, right_y)
    
    left_Lane_X_upperbound, left_Lane_X_lowerbound, right_Lane_X_upperbound, right_Lane_X_lowerbound = get_fitLane(left_slope, left_intercept
                                                                                                                    , right_slope, right_intercept
                                                                                                                    , Lane_Y_lowerbound
                                                                                                                    , Lane_Y_upperbound)
    if left_Lane_X_upperbound > right_Lane_X_lowerbound:
        left_Lane_X_upperbound = (right_intercept - left_intercept)/(left_slope - right_slope)
        right_Lane_X_lowerbound = left_Lane_X_upperbound
        Lane_Y_lowerbound = left_Lane_X_upperbound * left_slope + left_intercept
    
    
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, [[left_Lane_X_upperbound, Lane_Y_lowerbound, left_Lane_X_lowerbound, Lane_Y_upperbound]
                          ,[right_Lane_X_upperbound, Lane_Y_upperbound, right_Lane_X_lowerbound, Lane_Y_lowerbound]])
    return line_img

def hough_line_filter(lines, filter_y_uppderbound):
    lines = lines[:,lines[1] > filter_y_uppderbound]
    
    # calculate the slope of the lines
    slopes = np.divide(lines[1]-lines[3], lines[0]-lines[2])
    
    # right lines: get the most top left point
    right_lines = (lines[:,np.array((slopes > 0.5))])
    right_ref_x, right_ref_y, _, _ = right_lines[:,0]
    
    # left lines: get the most top right point
    left_lines = (lines[:,np.array((slopes < -0.5))])
    left_max_x_arg = np.argmax(left_lines[2], axis=0)
    _, _, left_ref_x, left_ref_y = left_lines[:,left_max_x_arg]
    
    return right_lines, left_lines, (left_ref_x, left_ref_y), (right_ref_x, right_ref_y)

In [5]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [6]:
def process_image(img):
    imshape = img.shape

    gray_img = grayscale(img) # output an image with only one color channel

    low_threshold = 50
    high_threshold = 200
    canny_img = canny(gray_img, low_threshold, high_threshold)

    kernel_size = 1
    blurred_img = gaussian_blur(canny_img, kernel_size)
    vertices = np.array([[(0,imshape[0]),(450, imshape[0]//2), (490, imshape[0]//2), (imshape[1],imshape[0])]], dtype=np.int32)
    masked_image = region_of_interest(blurred_img, vertices)

    rho = 1
    theta = np.pi/180
    threshold = 15
    min_line_len = 40
    max_line_gap = 5
    filter_y_uppderbound = imshape[0]
    line_img = hough_lines(masked_image, rho, theta, threshold, min_line_len, max_line_gap, filter_y_uppderbound) 

    return weighted_img(line_img, img)

Let's try the one with the solid white lane on the right first ...

In [7]:
videoSaved_dir = "test_videos_output"
if not os.path.exists(videoSaved_dir):
    os.makedirs(videoSaved_dir)


white_output = videoSaved_dir + '/solidWhiteRight.mp4'

## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
# white_output.subclip(0,2)

## You may also uncomment the following line for a subclip of the first 5 seconds
clip1 = VideoFileClip("test_videos//solidWhiteRight.mp4").subclip(0,5)
# clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")

In [14]:
clip1.get_frame(1).shape

(540, 960, 3)

In [9]:
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
# %time white_clip.write_videofile(white_output, fps=10, audio=False)

In [10]:
%time white_clip.write_videofile("my_new_video.mp4", audio=False)

[MoviePy] >>>> Building video my_new_video.mp4
[MoviePy] Writing video my_new_video.mp4



  4%|█▋                                        | 5/126 [00:00<00:02, 48.04it/s]

ValueError: attempt to get argmax of an empty sequence


  4%|█▋                                        | 5/126 [00:13<05:22,  2.66s/it]

Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [10]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

## Improve the draw_lines() function

**At this point, if you were successful with making the pipeline and tuning parameters, you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform. As mentioned previously, try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines. You can see an example of the result you're going for in the video "P1_example.mp4".**

**Go back and modify your draw_lines function accordingly and try re-running your pipeline. The new output should draw a single, solid line over the left lane line and a single, solid line over the right lane line. The lines should start from the bottom of the image and extend out to the top of the region of interest.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [77]:
yellow_output = 'test_videos_output/solidYellowLeft.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4').subclip(0,5)
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/solidYellowLeft.mp4
[MoviePy] Writing video test_videos_output/solidYellowLeft.mp4


100%|███████████████████████████████████████████████████████████████████████████████▉| 681/682 [00:21<00:00, 30.99it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidYellowLeft.mp4 

Wall time: 22.4 s


In [78]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Writeup and Submission

If you're satisfied with your video outputs, it's time to make the report writeup in a pdf or markdown file. Once you have this Ipython notebook ready along with the writeup, it's time to submit for review! Here is a [link](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) to the writeup template file.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [ ]:
challenge_output = 'test_videos_output/challenge.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip3 = VideoFileClip('test_videos/challenge.mp4').subclip(0,5)
clip3 = VideoFileClip('test_videos/challenge.mp4')
challenge_clip = clip3.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))